In [ ]:
import cv2
import os 
import numpy as np 
import math 
from matplotlib import pyplot as plt
import keyboard
import time
# function to display the coordinates of
# of the points clicked on the image

def GridGen(x , y , xn , yn):
      
    xb=np.linspace(x[0],x[1],xn)

    xt=np.linspace(x[3],x[2],xn)
    
    yr=np.linspace(y[1],y[2],yn)

    yl=np.linspace(y[0],y[3],yn)
    
    xm = np.linspace(0,1,xn)
    ym = np.linspace(0,1,yn)
    
    X = np.zeros((yn,xn))
    Y = np.zeros((yn,xn))
    
    for j in range(yn): 
        for i in range(xn):
            Y[j][i]= (1-xm[i])*yl[j] + xm[i]*yr[j]
            X[j][i]= (1-ym[j])*xb[i] + ym[j]*xt[i]
    return X,Y

def GridLis(xn,yn):
    masterlis = []
    templis = []    
    xb0 = 0
    xb1 = np.shape(gray)[1]
    xt0 = 0
    xt1 = np.shape(gray)[1]
    ybl = np.shape(gray)[0]-Dif
    ybr = np.shape(gray)[0]-Dif
    ytr = Dif
    ytl = Dif
    x = [xb0,xb1,xt1,xt0] #[xb0,xb1,xt0,xt1]
    y = [ybl,ybr,ytr,ytl] #[ybl,ybr,ytr,ytl]
    xnode = xn
    ynode = yn
    X1,Y1 = GridGen(x,y,xnode,ynode)
    #print(np.shape(X1))
    xt = len(X1[0][:])
    yt = (np.shape(X1)[0])
    xr = int(np.round(X1[0][1]-X1[0][0]))
    yr = int(np.round(Y1[0][0]-Y1[1][0]))
    #print(xr,yr)
    for n in range(yt-1):
        for k in range(xt-1):
            for i in range(xr):
                for j in range(yr):
                    templis.append([int(X1[n][k])+i,int(Y1[n][k])-j])
            #print(np.shape(templis))
            masterlis.append(templis)
            templis=[]
    
    return masterlis
            
def AutoFill(grid,im,FillList):
    
    for i in range(len(grid)):
        tot = True
        for j in grid[i]:
            x,y=j
            if im[y,x]!=255:
                tot =False
        if tot == True:
            FillList[i]=1
    return FillList

def search(grid,coord):
    index = 0
    for i in range(len(grid)):
        for j in range(len(grid[0,:])):
            #print(test[i,j],coord)
            if np.array_equal(grid[i,j],coord):
                index = i
    return index
def click_event(event, x, y, flags, params):
    global tclis
    tc0=time.time()
    # checking for left mouse clicks
    if event == cv2.EVENT_LBUTTONDOWN:
        
        coords=np.array([x,y])
        index = search(MasterGrid,coords)
        #print(index)
        if int(FillList[index])==1:
            #print('clear')
            FillList[index]=0
        elif int(FillList[index])==0:
            #print('Fill')
            FillList[index]=1
        for i in range(len(MasterGrid)):
            if FillList[i]==1:
                for j in MasterGrid[i]:
                    x,y=j
                    img[y,x]= 255
            if FillList[i]==0:
                for j in MasterGrid[i]:
                    x,y=j
                    img[y,x]= grayog[y,x]
        
        cv2.imshow('image',img)
        tc=time.time()-tc0
        tclis.append(tc)
        #print(coords)
             
            
    
# main Algorithm
if __name__=="__main__":
    
    video = cv2.VideoCapture('FF_T1_Cu1200_R245fa_20C_50kW_Ref300_2000fps_2019_08_19a.avi')
    frames = []
    Chosenframes =[]
    i=0
    while(video.isOpened()):
        ret, frame = video.read()
        if ret == False:
            break
    #cv2.imwrite('frame'+str(i)+'.jpg',frame)
        frames.append(frame)
        i+=1
    #for i in np.linspace(0,len(frames)-1,3): ## autoselect samples
    #    Chosenframes.append(frames[int(i)])
    Chosenframes =[frames[0],frames[75],frames[-1]] # manual select
    timeLis = []
    frameLis = []
    tclis = []
    video.release()
    cv2.destroyAllWindows
    k = 0
    for j in Chosenframes:
        autofill =[]
        t0 = time.time()
        print('Frame {}'.format(k))
        image = j
        crop = np.array(image.copy())
        Kernel = (7,7)    
        KernelSize = 7    
        morphKernal = (15,15)
        height = len(image) 
        Dif = 235
        crop[:,:,(0,2)] = 0  
        crop[0: Dif] = 0 
        crop[height-Dif: height] = 0

        gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)  
        equal = cv2.equalizeHist(gray) 
        image = cv2.GaussianBlur(equal, Kernel, 0)
        edged = cv2.Canny(image, 45, 170,KernelSize)  
        edg = edged.copy()
        mask = np.zeros((1024,1280),np.uint8)
        kerneld = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,morphKernal)
        dilated = cv2.dilate(edg, kerneld)
        #kerneld2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,morphKernal)
        #dilated2 = cv2.dilate(edg, kerneld2)
        cnts, _ = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(gray,cnts,-1,(255,255,255),1)
        cv2.drawContours(mask,cnts,-1,(255,255,255),-1)
        for i in range(1024):
            for j in range(1280):
                if mask[i,j] == 255:
                    autofill.append(1)
        grayarea = len(gray)*1280-235*1280*2
        area = (len(autofill))/grayarea
        ta = time.time()-t0
        print('Auto Time {}'.format(ta))
        print('Auto Area')
        print(area,len(autofill))       
        #print(np.shape(mask))
        #print(mask[228,16])
        
        plt.figure()
        plt.imshow(mask,cmap='gray')
        # reading the image
        MasterGrid = np.array(GridLis(300,120))
        FillList = np.zeros(len(MasterGrid))
        FillList = AutoFill(MasterGrid,mask,FillList)
        img = gray
        grayog = gray.copy()
        # displaying the image
        cv2.imshow('image', gray)

        # setting mouse handler for the image
        # and calling the click_event() function
        cv2.setMouseCallback('image', click_event)
        #print('next')
        # wait for a key to be pressed to exit
        cv2.waitKey(0)
        #print('next2')
        #keyboard.wait('esc')
        # close the window
        cv2.destroyAllWindows()
        t1=time.time()-t0
        timeLis.append(t1)
        frameLis.append(k)
        tca = np.mean(tclis)
        print("Average Time per Fill/Clear {} s".format(tca))
        print("Time per frame {} s".format(t1))
        tclis = []
        k+=1
        dry = len(FillList)-np.count_nonzero(FillList == 1)
        perc = 1-dry/len(FillList)
        print("Percentage Dryout Area {}".format(perc))
    plt.figure()
    plt.plot(frameLis,timeLis,'k.')
    plt.grid()
    plt.show()